In [1]:
import pandas as pd
import numpy as np
import time
import csv
import math
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
start = time.time()

C_Lambda = 0.01 #Regularization constant
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10
M = 6 #number of basis functions decided based on Kmeans clustering.
PHI = []
IsSynthetic = False

# Create Human Observed Dataset

In [3]:
def Create_Human_Observe_Dataset():
    
    d = []
    with open('HumanObserved-Features-Data.csv') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            d.append(row)

    dataset = {l['img_id']: [v for k, v in l.items() if k not in ('', 'img_id')] for l in d}
    #print(dataset)   
    samepairs = []
    SP_target = []
    with open('same_pairs.csv') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            samepairs.append(row[:2])
            SP_target.append(row[2])
        #print(samepairs)
        #print(SP_target)
        for i in range(len(samepairs)):
            for k,v in dataset.items():
                
               # print(k,v)
                if(samepairs[i][0])==k:
                    samepairs[i].append(v)
                if(samepairs[i][1])==k:
                    samepairs[i].append(v)
        #print(samepairs)
    #print(len(samepairs))

    d=[]
    for i in range(len(samepairs)-1): #792 rows
        for j in range(len(samepairs[1])-2): #Add first 3 columns directly.
            #print(i+1,j)
            #print(a[i+1][j])
            d.append(samepairs[i+1][j])
        for k in range(9):
                #print(samepairs[i+1][3][k])
                #print("------")

                d.append(int(samepairs[i+1][2][k]))
        for k in range(9):
            #print(samepairs[i+1][4][k])
            d.append(int(samepairs[i+1][3][k]))
    #print(d)

    e=[]
    for i in range(len(samepairs)-1): #792 rows
        for j in range(len(samepairs[1])-2): #Add first 3 columns directly.
            #print(i+1,j)
            #print(a[i+1][j])
            e.append(samepairs[i+1][j])
        for k in range(9):
                #print(samepairs[i+1][3][k])
                #print("------")

                e.append(abs(int(samepairs[i+1][2][k])-int(samepairs[i+1][3][k])))

    #print(e)


    SP = np.array(d).reshape(791,20)
    #print(SP)

    #print((SP[0]))

    Sub_SP = np.array(e).reshape(791,11)
    #print(Sub_SP)

    diff_pairs = pd.DataFrame(pd.read_csv("diffn_pairs.csv"))
    diff_pairs.tail()
    diff_pairs.describe()

    different_pairs = []
    DP_target=[]
    with open('diffn_pairs.csv') as csvfile:
        diff_reader = csv.reader(csvfile)
        for row in diff_reader:
            different_pairs.append(row[:2])
            DP_target.append(row[2])
    #print(len(different_pairs))
    #print(len(DP_target))

    #print((different_pairs[293032]))

    for i in range(len(different_pairs)):
        for k,v in dataset.items():
           # print(k,v)
            if(different_pairs[i][0])==k:
                different_pairs[i].append(v)
            if(different_pairs[i][1])==k:
                different_pairs[i].append(v)
    #print(different_pairs)
    #print(len(different_pairs))

    #print((different_pairs[1]))

    c=[]
    for i in range(len(different_pairs)-1): #293031 rows
        for j in range(len(different_pairs[1])-2): #Add first 3 columns directly.
            #print(i+1,j)
            #print(a[i+1][j])
            c.append(different_pairs[i+1][j])
        for k in range(9):
                #print(samepairs[i+1][3][k])
                #print("------")

                c.append(int(different_pairs[i+1][2][k]))
        for k in range(9):
            #print(samepairs[i+1][4][k])
            c.append(int(different_pairs[i+1][3][k]))
    #print(c)

    f=[]
    for i in range(len(different_pairs)-1): #792 rows
        for j in range(len(samepairs[1])-2): #Add first 3 columns directly.
            #print(i+1,j)
            #print(a[i+1][j])
            f.append(different_pairs[i+1][j])
        for k in range(9):
                #print(samepairs[i+1][3][k])
                #print("------")

                f.append(abs(int(different_pairs[i+1][2][k])-int(different_pairs[i+1][3][k])))

    print(f)

    Diff_pairs = np.array(c).reshape(293032,20)
    DP = Diff_pairs[0:791,]
    Unseen_DP = Diff_pairs[791:,]
    DP.shape

    Sub_DP = np.array(f).reshape(293032,11)
    Sub_DP = Sub_DP[0:791,]
    Sub_DP.shape

    Human_Observed_Dataset = np.concatenate((SP,DP), axis = 0)
    Human_Observed_Dataset.shape
    HODS = Human_Observed_Dataset[:, 2:]
    HODS.shape

    Sub_Human_Observed_Dataset = np.concatenate((Sub_SP,Sub_DP), axis = 0)
    Sub_Human_Observed_Dataset.shape

    #print(time.time()-start)
    SHODS = Sub_Human_Observed_Dataset[:, 2:]
        
        
    target = np.concatenate((np.array(SP_target[1:]), np.array(DP_target[1:792])), axis=0)
    t = target.reshape(-1,1)
    #print(t.shape)
    HODS.shape
    Final_HODS = np.concatenate((HODS, t), axis=1)
   # print(Final_HODS.shape)
    #print(Final_HODS)
    Final_SHODS = np.concatenate((SHODS, t), axis=1)
    Final_SHODS.shape

    #Convert str to int
    temp = Final_HODS.reshape(-1)
    temp.shape
    List = temp.tolist()
    List = [int(x) for x in List]
    Final_HODS = np.array(List).reshape(1582,19)
    temp1 = Final_SHODS.reshape(-1)
    temp1.shape
    List = temp1.tolist()
    List = [int(x) for x in List]
    Final_SHODS = np.array(List).reshape(1582,10)
    #print(Final_SHODS.dtype)
    print("Human Observed Dataset Created")
    return Final_HODS, Final_SHODS
  

In [4]:
def Select_HODS(Final_HODS, Final_SHODS, Concatenated = True):  #Select the dataset as per concatenation/subtraction
      if Concatenated == True:  
        print("Human Observed Dataset with Concatenation selected")
        np.random.shuffle(Final_HODS)
        #print(Final_HODS)
        training_data = Final_HODS[:, :18]
        print(training_data.shape)
        target_data = Final_HODS[:, 18:]
        #print(target_data.shape)
        #print(len(training_data[0]))
        RawData = np.transpose(training_data)
        RawTarget = target_data
        return RawData, RawTarget

      else:
            print("Human Observed Dataset with Subtraction selected")
            np.random.shuffle(Final_SHODS)
            #print(Final_SHODS)
            training_data = Final_SHODS[:, :9]
            #print(training_data.shape)
            target_data = Final_SHODS[:, 9:]
            #print(target_data.shape)
            #print(len(training_data[0]))
            RawData = np.transpose(training_data)
            RawTarget = target_data
            #print(RawData.shape)
            return RawData, RawTarget

In [5]:
def Generate_Data_Split(RawData, RawTarget): #Split the data into Training, Validation, Testing
    TrainingTarget = (GenerateTrainingTarget(RawTarget,TrainingPercent).reshape(-1))
    TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
    #print(TrainingTarget.shape)
    #print(TrainingData.shape)

    ValDataAct = (GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))).reshape(-1))
    ValData    = GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))
    #print(ValDataAct.shape)
    #print(ValData.shape)

    TestDataAct = (GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))).reshape(-1))
    TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
    #print(ValDataAct.shape)
    #print(ValData.shape)

    return TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct

# HUMAN OBSERVED DATASET

In [6]:
#Create Human Observed Dataset
Final_HODS, Final_SHODS = Create_Human_Observe_Dataset()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Human Observed Dataset Created


# GSC DATASET

In [40]:
def create_gsc_dataset():
    d = []
    with open('GSC-Features.csv') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            d.append(row)

    dataset = {l['img_id']: [v for k, v in l.items() if k not in ('', 'img_id')] for l in d}
    #print(dataset)   

    #for k,v in dataset.items():
     #   print(k,v)

    same_pairs = pd.DataFrame(pd.read_csv("GSC-Features.csv"))
    same_pairs.describe()

    GSC_samepairs = []
    GSC_SP_target = []
    with open('GSC_same_pairs.csv') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            GSC_samepairs.append(row[:2])
            GSC_SP_target.append(row[2])
       # print(len(GSC_samepairs))
        #print(len(GSC_SP_target))

    #for i in range(len(samepairs)):
        #print(samepairs[i+1])
    #print(GSC_samepairs[1])

    for i in range(5000):
        for k,v in dataset.items():
            #print(k,v)
            if(GSC_samepairs[i][0])==k:
                GSC_samepairs[i].append(v)
            if(GSC_samepairs[i][1])==k:
                GSC_samepairs[i].append(v)
    #print(samepairs)
    print("length of gsc same pairs",len(GSC_samepairs[0:5000]))

    #print((GSC_samepairs[1]))


    d=[]
    for i in range(4999): #792 rows
        for j in range(len(GSC_samepairs[1])-2): #Add first 2 columns directly.
            #print(i+1,j)
            #print(a[i+1][j])
            d.append(GSC_samepairs[i+1][j])
        for k in range(512):
                #print(samepairs[i+1][3][k])
                #print("------")

                d.append(int(GSC_samepairs[i+1][2][k]))
        for k in range(512):
            #print(samepairs[i+1][4][k])
            d.append(int(GSC_samepairs[i+1][3][k]))
    #print("d",len(d))

    e=[]
    for i in range(4999): #792 rows
        for j in range(len(GSC_samepairs[1])-2): #Add first 3 columns directly.
            #print(i+1,j)
            #print(a[i+1][j])
            e.append(GSC_samepairs[i+1][j])
        for k in range(512):
                #print(samepairs[i+1][3][k])
                #print("------")

                e.append(abs(int(GSC_samepairs[i+1][2][k])-int(GSC_samepairs[i+1][3][k])))

    #print(e)

    import numpy as np
    GSC_SP = np.array(d).reshape(4999,1026)
    print("GSC_SP",GSC_SP.shape)

    #print((GSC_SP[0]))

    GSC_Sub_SP = np.array(e).reshape(4999,514)
    print("GSC_SUB_SP",GSC_Sub_SP.shape)

    diff_pairs = pd.DataFrame(pd.read_csv("GSC_diffn_pairs.csv"))
    diff_pairs.tail()
    diff_pairs.describe()

    GSC_different_pairs = []
    GSC_DP_target=[]
    with open('GSC_diffn_pairs.csv') as csvfile:
        diff_reader = csv.reader(csvfile)
        for row in diff_reader:
            GSC_different_pairs.append(row[:2])
            GSC_DP_target.append(row[2])
    #print(len(GSC_different_pairs))
    #print(len(GSC_DP_target))

    #print((GSC_different_pairs[1]))

    for i in range(5000):
        for k,v in dataset.items():
           # print(k,v)
            if(GSC_different_pairs[i][0])==k:
                GSC_different_pairs[i].append(v)
            if(GSC_different_pairs[i][1])==k:
                GSC_different_pairs[i].append(v)
    #print(different_pairs)
    #print(len(different_pairs[0:1000]))



    c=[]
    for i in range(4999): #293031 rows
        for j in range(len(GSC_different_pairs[1])-2): #Add first 3 columns directly.
            #print(i+1,j)
            #print(a[i+1][j])
            c.append(GSC_different_pairs[i+1][j])
        for k in range(512):
                #print(samepairs[i+1][3][k])
                #print("------")

                c.append(int(GSC_different_pairs[i+1][2][k]))
        for k in range(512):
            #print(samepairs[i+1][4][k])
            c.append(int(GSC_different_pairs[i+1][3][k]))
    #print(c)

    #print("c",len(c))

    f=[]
    for i in range(4999): #792 rows
        for j in range(len(GSC_different_pairs[1])-2): #Add first 3 columns directly.
            #print(i+1,j)
            #print(a[i+1][j])
            f.append(GSC_different_pairs[i+1][j])
        for k in range(512):
                #print(samepairs[i+1][3][k])
                #print("------")

                f.append(abs(int(GSC_different_pairs[i+1][2][k])-int(GSC_different_pairs[i+1][3][k])))

    #print("f", len(f))

    GSC_DP = np.array(c).reshape(4999,1026)
    print("GSC_DP",GSC_DP.shape)

    GSC_Sub_DP = np.array(f).reshape(4999,514)
    print("GSC_SUB_DP",GSC_Sub_DP.shape)

    GSC_Dataset = np.concatenate((GSC_SP[:2500],GSC_DP[:2500]), axis = 0)
    GSC_Dataset = GSC_Dataset[:, 2:]
    print("GSC_Dataset", GSC_Dataset.shape)

    Unseen_GSC = np.concatenate((GSC_SP[2500:], GSC_DP[2500:]), axis = 0)
    Unseen_GSC = Unseen_GSC[:, 2:]
    print("unseen_gsc", Unseen_GSC.shape)

    Sub_GSC_Dataset = np.concatenate((GSC_Sub_SP[:2500],GSC_Sub_DP[:2500]), axis = 0)
    Sub_GSC_Dataset = Sub_GSC_Dataset[:, 2:]
    print("Sub_gsc_dataset", Sub_GSC_Dataset.shape)

    Sub_Unseen_GSC = np.concatenate((GSC_Sub_SP[2500:], GSC_Sub_DP[2500:]), axis=0)
    Sub_Unseen_GSC = Sub_Unseen_GSC[:, 2:]
    print("Unseen_Sub_gsc", Sub_Unseen_GSC.shape)


    GSC_target = np.concatenate((GSC_SP_target[1:2501],GSC_DP_target[1:2501]), axis=0)
    GSC_target = GSC_target.reshape(-1,1)
    print("GSC_target", GSC_target.shape)

    Unseen_GSC_target = np.concatenate((GSC_SP_target[2500:4999],GSC_DP_target[2500:4999]), axis=0)
    Unseen_GSC_target = Unseen_GSC_target.reshape(-1,1)
    print("unseen_target", Unseen_GSC_target.shape) 

    GSC_Final = np.concatenate((GSC_Dataset, GSC_target), axis=1)
    print("GSC_final", GSC_Final.shape)

    GSC_Sub_Final = np.concatenate((Sub_GSC_Dataset, GSC_target), axis=1)
    print("GSC_sub_final", GSC_Sub_Final.shape )

    #Generate unseen data
    Unseen_GSC_Final = np.concatenate((Unseen_GSC, Unseen_GSC_target), axis=1)
    print("Unseen_GSC_final", Unseen_GSC_Final.shape)

    Unseen_GSC_Sub_Final = np.concatenate((Sub_Unseen_GSC, Unseen_GSC_target), axis=1)
    print("Unseen_GSC_sub_final", Unseen_GSC_Sub_Final.shape)
    
    #Convert str to int
    temp = GSC_Final.reshape(-1)
    #temp.shape
    List = temp.tolist()
    List = [int(x) for x in List]
    GSC_Final = np.array(List).reshape(5000,1025)
    temp1 = GSC_Sub_Final.reshape(-1)
    #temp1.shape
    List = temp1.tolist()
    List = [int(x) for x in List]
    GSC_Sub_Final = np.array(List).reshape(5000,513)
    temp2 = Unseen_GSC_Final.reshape(-1)
    temp2.shape
    List = temp2.tolist()
    List = [int(x) for x in List]
    Unseen_GSC_Final = np.array(List).reshape(4998,1025)
    temp3 = Unseen_GSC_Sub_Final.reshape(-1)
    temp3.shape
    List = temp3.tolist()
    List = [int(x) for x in List]
    Unseen_GSC_Sub_Final = np.array(List).reshape(4998,513)

    
    return GSC_Final, GSC_Sub_Final


In [41]:
def Select_GSC(GSC_Final, GSC_Sub_Final, Concatenated = True): #Select GSC dataset - Concatenation/Subtraction
    if Concatenated == True:
        print("GSC Dataset with feature concatenation")
        #GSC CONCATENATED
        np.random.shuffle(GSC_Final)
        #print(GSC_Final)
        training_data = GSC_Final[:, :1024]
        #print(training_data.shape)
        target_data = GSC_Final[:, 1024:]
        #print(target_data.shape)
        #print(len(training_data[0]))
        
        RawData = np.transpose(training_data)
        RawTarget = target_data
        #print(RawData[500])

        cols = []
        var = []
        for i in range(len(RawData)):
            Var = np.var(RawData[i], axis=0)
            var.append(Var)
            if Var == 0 :
                cols.append(i)
        #print(cols)
        #print(len(cols))
        #print(var)

        RawData = np.delete(RawData, cols, axis=0)
        print(RawData.shape)
        RawData = RawData + 0.00001*np.random.rand(RawData.shape[0], RawData.shape[1])
        return RawData, RawTarget
        
    else:
        print("GSC Dataset with feature subtraction")
        #GSC SUBTRACTED
        np.random.shuffle(GSC_Sub_Final)
        #print(GSC_Sub_Final)
        training_data = GSC_Sub_Final[:, :512]
        #print(training_data.shape)
        target_data = GSC_Sub_Final[:, 512:]
        #print(target_data.shape)
        #print(len(training_data[0]))
        
        RawData = np.transpose(training_data)
        RawTarget = target_data
        #print(RawData[500])

        cols = []
        var = []
        for i in range(len(RawData)):
            Var = np.var(RawData[i], axis=0)
            var.append(Var)
            if Var == 0 :
                cols.append(i)
        #print(cols)
        #print(len(cols))
        #print(var)

        RawData = np.delete(RawData, cols, axis=0)
        print(RawData.shape)
        RawData = RawData + 0.00001*np.random.rand(RawData.shape[0], RawData.shape[1])
        return RawData, RawTarget

In [42]:
GSC_Final, GSC_Sub_Final = create_gsc_dataset()

length of gsc same pairs 5000
GSC_SP (4999, 1026)
GSC_SUB_SP (4999, 514)
GSC_DP (4999, 1026)
GSC_SUB_DP (4999, 514)
GSC_Dataset (5000, 1024)
unseen_gsc (4998, 1024)
Sub_gsc_dataset (5000, 512)
Unseen_Sub_gsc (4998, 512)
GSC_target (5000, 1)
unseen_target (4998, 1)
GSC_final (5000, 1025)
GSC_sub_final (5000, 513)
Unseen_GSC_final (4998, 1025)
Unseen_GSC_sub_final (4998, 513)


In [7]:
def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80): #training data split(80%)
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    #print(T_len)
    d2 = rawData[:,0:T_len]
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2

def GenerateTrainingTarget(rawTraining,TrainingPercent = 80): #training target split
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    #print(TrainingLen)
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t

def GenerateValData(rawData, ValPercent, TrainingCount): #Generate validation data set as with 10% split
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Data Generated..")  
    return dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount): #Validation target split 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t
def GenerateBigSigma(Data, MuMatrix,TrainingPercent,IsSynthetic): #Generates a Diagonal matrix(41,41) of variances of all features
    BigSigma    = np.zeros((len(Data),len(Data)))
    #print (BigSigma)
    DataT       = np.transpose(Data)
    #print (DataT)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))        
    varVect     = []
    for i in range(0,len(DataT[0])):
        vct = []
        for j in range(0,int(TrainingLen)):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct)) #calculating variance of data.
        
    
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]
    if IsSynthetic == True:
        BigSigma = np.dot(3,BigSigma)
    else:
        BigSigma = np.dot(200,BigSigma)
    ##print ("BigSigma Generated..")
    #print(BigSigma)
    return BigSigma

def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow) #returns (M,41) matrix
    T = np.dot(BigSigInv,np.transpose(R))  #returns (41,M) matrix
    L = np.dot(R,T) #computes dot product of the two matrices
    return L

def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv)) #(M,41).(41,41).(41,M) matrices as per the radial basis formula
    return phi_x

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.inv(BigSigma) #inverse the big sigma matrix
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    #print ("PHI Generated..")
    return PHI #returns matrix of gaussian radial function.

def GetWeightsClosedForm(PHI, T, Lambda): #Generates the weights as per the closed form formula
    Lambda_I = np.identity(len(PHI[0])) #I matrix from the closed form formula
    for i in range(0,len(PHI[0])): 
        Lambda_I[i][i] = Lambda #Lambda matrix from the formula
    PHI_T       = np.transpose(PHI) #PHI from the forumla
    PHI_SQR     = np.dot(PHI_T,PHI) # PHI . PHI transpose as seen in the formula
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR) # PHI. PHI transpose + Lambda*I 
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI) # Inverse of PHI. PHI transpose + Lambda*I 
    INTER       = np.dot(PHI_SQR_INV, PHI_T) #Inverse of (PHI. PHI transpose + Lambda*I) . PHI Transpose
    W           = np.dot(INTER, T) ##Inverse of (PHI. PHI transpose + Lambda*I) . PHI Transpose . T "Closed form equation"
    ##print ("Training Weights Generated..")
    return W

def GetValTest(VAL_PHI,W): #Peforms prediction to get final output.
    Y = np.dot(W,np.transpose(VAL_PHI))
    ##print ("Test Out Generated..")
    return Y

def GetErms(VAL_TEST_OUT,ValDataAct): #Calculates RMS values of given data set.
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2) #sum of squared errors.
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    ##print ("Accuracy Generated..")
    ##print ("Validation E_RMS : " + str(math.sqrt(sum/len(VAL_TEST_OUT))))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

# Closed Form Solution [Finding Weights using Moore- Penrose pseudo- Inverse Matrix] 

In [73]:
def Linear_Regression(TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct, Concatenate = True):
    ErmsArr = []
    AccuracyArr = []
    X_kmeans = np.transpose(TrainingData)
    kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingData))
    #print (kmeans)
    y_kmeans = kmeans.predict(X_kmeans)
    #print (len(y_kmeans))
    Mu = kmeans.cluster_centers_ #find the center of clusters from traning data
    #print(Mu)
    BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent,IsSynthetic)
    TRAINING_PHI = GetPhiMatrix(RawData, Mu, BigSigma, TrainingPercent)
    W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
    TEST_PHI     = GetPhiMatrix(TestData, Mu, BigSigma, 100) 
    VAL_PHI      = GetPhiMatrix(ValData, Mu, BigSigma, 100)

    #print(Mu.shape)
    #print(BigSigma.shape)
    #print(TRAINING_PHI.shape)
    #print(W.shape)
    #print(VAL_PHI.shape)
    #print(TEST_PHI.shape)

    TR_TEST_OUT  = GetValTest(TRAINING_PHI,W)
    VAL_TEST_OUT = GetValTest(VAL_PHI,W)
    TEST_OUT     = GetValTest(TEST_PHI,W)
    #print(TEST_OUT)
    TestAcc = []
    TrainingAccuracy   = str(GetErms(TR_TEST_OUT,TrainingTarget))
    ValidationAccuracy = str(GetErms(VAL_TEST_OUT,ValDataAct))
    TestAccuracy       = str(GetErms(TEST_OUT,TestDataAct))
    #TestAcc.append[TestAccuracy]
    #print (TestAccuracy)

    W_Now        = np.dot(220, W)
    La           = 0.5 #Regularization constant
    learningRate = 0.2
    L_Erms_Val   = []
    L_Erms_TR    = []
    L_Erms_Test  = []
    W_Mat        = []

    for i in range(0,1000):

        #print ('---------Iteration: ' + str(i) + '--------------')
        Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
        La_Delta_E_W  = np.dot(La,W_Now)
        Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
        Delta_W       = -np.dot(learningRate,Delta_E)
        W_T_Next      = W_Now + Delta_W
        W_Now         = W_T_Next

        #-----------------TrainingData Accuracy---------------------#
        TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
        Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
        L_Erms_TR.append(float(Erms_TR.split(',')[1]))

        #-----------------ValidationData Accuracy---------------------#
        VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
        Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
        L_Erms_Val.append(float(Erms_Val.split(',')[1]))

        #-----------------TestingData Accuracy---------------------#
        GD_TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
        Erms_Test = GetErms(GD_TEST_OUT,TestDataAct)
        L_Erms_Test.append(float(Erms_Test.split(',')[1]))

    print ('----------Gradient Descent Solution--------------------')
    if (Concatenate == True):
        print("E_rms for Human Observed Dataset with feature concatenation")
        #print ("Lambda  = 0.0001\neta=0.01")
        print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
        print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
        print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))
    else:
        print("E_rms for Human Observed Dataset with feature subtraction")
        #print ("Lambda  = 0.0001\neta=0.01")
        print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
        print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
        print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))
        
   

In [74]:
#HUMAN OBSERVED DATASET
RawData, RawTarget = Select_HODS(Final_HODS, Final_SHODS, Concatenated = True)
#print(RawData.shape, RawTarget.shape)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
Linear_Regression(TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct, Concatenate = True)
#FeatureSubtraced Dataset
RawData, RawTarget = Select_HODS(Final_HODS, Final_SHODS, Concatenated = False)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
#print(RawData.shape, RawTarget.shape)
Linear_Regression(TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct, Concatenate = False)

Human Observed Dataset with Concatenation selected
(1582, 18)
----------Gradient Descent Solution--------------------
E_rms for Human Observed Dataset with feature concatenation
E_rms Training   = 0.35639
E_rms Validation = 0.36046
E_rms Testing    = 0.35066
Human Observed Dataset with Subtraction selected
----------Gradient Descent Solution--------------------
E_rms for Human Observed Dataset with feature subtraction
E_rms Training   = 0.45775
E_rms Validation = 0.42991
E_rms Testing    = 0.4342


In [75]:
#GSC DATASET
RawData, RawTarget = Select_GSC(GSC_Final, GSC_Sub_Final, Concatenated = True)
print(RawData.shape, RawTarget.shape)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
Linear_Regression(TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct, Concatenate = True)
#FeatureSubtraced Dataset
RawData, RawTarget = Select_GSC(GSC_Final, GSC_Sub_Final, Concatenated = False)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
#print(RawData.shape, RawTarget.shape)
Linear_Regression(TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct, Concatenate = False)

GSC Dataset with feature concatenation
(940, 5000)
(940, 5000) (5000, 1)
----------Gradient Descent Solution--------------------
E_rms for Human Observed Dataset with feature concatenation
E_rms Training   = 0.38227
E_rms Validation = 0.40908
E_rms Testing    = 0.38213
GSC Dataset with feature subtraction
(474, 5000)
----------Gradient Descent Solution--------------------
E_rms for Human Observed Dataset with feature subtraction
E_rms Training   = 0.41219
E_rms Validation = 0.42542
E_rms Testing    = 0.38851


# Neural Network

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
import numpy as np


drop_out = 0.2
first_dense_layer_nodes  = 256
hidden_dense_layer = 512
second_dense_layer_nodes = 1

def get_model(input_size):
    
    
    model = Sequential()
    
    model.add(Dense(first_dense_layer_nodes, input_dim=input_size))
    model.add(Activation('relu'))
    
    model.add(Dense(hidden_dense_layer))
    model.add(Activation('relu')) 
    
    # Why dropout?
    model.add(Dropout(drop_out))
    
    model.add(Dense(second_dense_layer_nodes))
    model.add(Activation('sigmoid'))
    
    model.summary()
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [24]:
def Neural_Net(TrainingData, TrainingTarget):
    validation_data_split = 0.2
    num_epochs = 1000
    model_batch_size = 128
    tb_batch_size = 32
    early_patience = 100

    tensorboard_cb   = TensorBoard(log_dir='logs', batch_size= tb_batch_size, write_graph= True)
    earlystopping_cb = EarlyStopping(monitor='val_loss', verbose=1, patience=early_patience, mode='min')

    history = model.fit(TrainingData.T
                        , TrainingTarget
                        , validation_split=validation_data_split
                        , epochs=num_epochs
                        , batch_size=model_batch_size
                        , callbacks = [tensorboard_cb,earlystopping_cb]
                       )
    scores = model.evaluate(TrainingData.T, TrainingTarget)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    return history

In [29]:
def NN_Graph(history):
    %matplotlib inline
    df = pd.DataFrame(history.history)
    df.plot(subplots=True, grid=True, figsize=(10,15))


In [26]:
def NN_Accuracy(TestData, TestDataAct):
        wrong   = 0
        right   = 0

        for i,j in zip(TestData.T, TestDataAct):
            y = model.predict(np.array(i).reshape(-1,TestData.shape[0]))
            #predictedTestLabel.append(decodeLabel(y.argmax()))

            if j.argmax() == y.argmax():
                right = right + 1
            else:
                wrong = wrong + 1

        print("Errors: " + str(wrong), " Correct :" + str(right))

        print("Testing Accuracy: " + str(right/(right+wrong)*100))

In [27]:
#HUMAN OBSERVED DATASET CONCATENATED
RawData, RawTarget = Select_HODS(Final_HODS, Final_SHODS, Concatenated = True)
#print(RawData.shape, RawTarget.shape)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
model = get_model(TrainingData.shape[0])
history = Neural_Net(TrainingData, TrainingTarget)

NN_Graph(history)

NN_Accuracy(TestData, TestDataAct)

Human Observed Dataset with Concatenation selected
(1582, 18)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 256)               4864      
_________________________________________________________________
activation_10 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               131584    
_________________________________________________________________
activation_11 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 513       
_________________________________________________________________
activation_12 

1012/1012 [==============================] - 0s 91us/step - loss: 0.0086 - acc: 0.9990 - val_loss: 0.0675 - val_acc: 0.9803
Epoch 51/1000
1012/1012 [==============================] - 0s 91us/step - loss: 0.0085 - acc: 0.9980 - val_loss: 0.0625 - val_acc: 0.9843
Epoch 52/1000
1012/1012 [==============================] - 0s 94us/step - loss: 0.0084 - acc: 0.9990 - val_loss: 0.0595 - val_acc: 0.9882
Epoch 53/1000
1012/1012 [==============================] - 0s 96us/step - loss: 0.0078 - acc: 0.9980 - val_loss: 0.0618 - val_acc: 0.9843
Epoch 54/1000
1012/1012 [==============================] - 0s 90us/step - loss: 0.0074 - acc: 0.9980 - val_loss: 0.0730 - val_acc: 0.9803
Epoch 55/1000
1012/1012 [==============================] - 0s 96us/step - loss: 0.0081 - acc: 0.9980 - val_loss: 0.0643 - val_acc: 0.9843
Epoch 56/1000
1012/1012 [==============================] - 0s 90us/step - loss: 0.0078 - acc: 0.9980 - val_loss: 0.0585 - val_acc: 0.9921
Epoch 57/1000
1012/1012 [=======================

1012/1012 [==============================] - 0s 86us/step - loss: 8.2286e-04 - acc: 1.0000 - val_loss: 0.0867 - val_acc: 0.9803
Epoch 110/1000
1012/1012 [==============================] - 0s 79us/step - loss: 8.6736e-04 - acc: 1.0000 - val_loss: 0.0738 - val_acc: 0.9803
Epoch 111/1000
1012/1012 [==============================] - 0s 77us/step - loss: 6.7989e-04 - acc: 1.0000 - val_loss: 0.0682 - val_acc: 0.9921
Epoch 112/1000
1012/1012 [==============================] - 0s 86us/step - loss: 6.0903e-04 - acc: 1.0000 - val_loss: 0.0783 - val_acc: 0.9803
Epoch 113/1000
1012/1012 [==============================] - 0s 89us/step - loss: 6.7101e-04 - acc: 1.0000 - val_loss: 0.0818 - val_acc: 0.9803
Epoch 114/1000
1012/1012 [==============================] - 0s 80us/step - loss: 6.7765e-04 - acc: 1.0000 - val_loss: 0.0739 - val_acc: 0.9803
Epoch 115/1000
1012/1012 [==============================] - 0s 75us/step - loss: 5.5530e-04 - acc: 1.0000 - val_loss: 0.0830 - val_acc: 0.9803
Epoch 116/1000

1012/1012 [==============================] - 0s 80us/step - loss: 2.3274e-04 - acc: 1.0000 - val_loss: 0.0925 - val_acc: 0.9803
Epoch 167/1000
1012/1012 [==============================] - 0s 94us/step - loss: 2.6355e-04 - acc: 1.0000 - val_loss: 0.0877 - val_acc: 0.9803
Epoch 168/1000
1012/1012 [==============================] - 0s 95us/step - loss: 2.3999e-04 - acc: 1.0000 - val_loss: 0.0771 - val_acc: 0.9843
Epoch 169/1000
1012/1012 [==============================] - 0s 90us/step - loss: 2.4442e-04 - acc: 1.0000 - val_loss: 0.0840 - val_acc: 0.9803
Epoch 170/1000
1012/1012 [==============================] - 0s 75us/step - loss: 2.0911e-04 - acc: 1.0000 - val_loss: 0.0820 - val_acc: 0.9803
Epoch 171/1000
1012/1012 [==============================] - 0s 80us/step - loss: 2.0508e-04 - acc: 1.0000 - val_loss: 0.0843 - val_acc: 0.9803
Epoch 172/1000
1012/1012 [==============================] - 0s 88us/step - loss: 1.9570e-04 - acc: 1.0000 - val_loss: 0.0845 - val_acc: 0.9803
Epoch 173/1000

In [35]:
#HUMAN OBSERVED DATASET SUBTRACTED
RawData, RawTarget = Select_HODS(Final_HODS, Final_SHODS, Concatenated = False)
#print(RawData.shape, RawTarget.shape)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
model = get_model(TrainingData.shape[0])
history = Neural_Net(TrainingData, TrainingTarget)

NN_Graph(history)
NN_Accuracy(TestData, TestDataAct)

Human Observed Dataset with Subtraction selected
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 256)               2560      
_________________________________________________________________
activation_16 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               131584    
_________________________________________________________________
activation_17 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 513       
_________________________________________________________________
activation_18 (Activation) 

1012/1012 [==============================] - 0s 102us/step - loss: 0.2062 - acc: 0.9180 - val_loss: 0.3292 - val_acc: 0.8465
Epoch 51/1000
1012/1012 [==============================] - 0s 96us/step - loss: 0.2028 - acc: 0.9180 - val_loss: 0.3203 - val_acc: 0.8622
Epoch 52/1000
1012/1012 [==============================] - 0s 99us/step - loss: 0.2041 - acc: 0.9130 - val_loss: 0.3318 - val_acc: 0.8622
Epoch 53/1000
1012/1012 [==============================] - 0s 101us/step - loss: 0.2037 - acc: 0.9150 - val_loss: 0.3302 - val_acc: 0.8622
Epoch 54/1000
1012/1012 [==============================] - 0s 102us/step - loss: 0.2024 - acc: 0.9160 - val_loss: 0.3295 - val_acc: 0.8622
Epoch 55/1000
1012/1012 [==============================] - 0s 94us/step - loss: 0.1966 - acc: 0.9190 - val_loss: 0.3308 - val_acc: 0.8504
Epoch 56/1000
1012/1012 [==============================] - 0s 101us/step - loss: 0.1950 - acc: 0.9150 - val_loss: 0.3298 - val_acc: 0.8543
Epoch 57/1000
1012/1012 [===================

1012/1012 [==============================] - 0s 79us/step - loss: 0.1585 - acc: 0.9338 - val_loss: 0.4422 - val_acc: 0.8346
Epoch 110/1000
1012/1012 [==============================] - 0s 86us/step - loss: 0.1599 - acc: 0.9269 - val_loss: 0.4355 - val_acc: 0.8386
Epoch 111/1000
1012/1012 [==============================] - 0s 94us/step - loss: 0.1600 - acc: 0.9279 - val_loss: 0.4378 - val_acc: 0.8425
Epoch 112/1000
1012/1012 [==============================] - 0s 101us/step - loss: 0.1646 - acc: 0.9249 - val_loss: 0.4369 - val_acc: 0.8386
Epoch 113/1000
1012/1012 [==============================] - 0s 87us/step - loss: 0.1602 - acc: 0.9338 - val_loss: 0.4444 - val_acc: 0.8425
Epoch 114/1000
1012/1012 [==============================] - 0s 97us/step - loss: 0.1724 - acc: 0.9229 - val_loss: 0.4229 - val_acc: 0.8346
Epoch 115/1000
1012/1012 [==============================] - 0s 96us/step - loss: 0.1507 - acc: 0.9377 - val_loss: 0.4379 - val_acc: 0.8583
Epoch 116/1000
1012/1012 [===============

In [44]:
#GSC CONCATENATED
RawData, RawTarget = Select_GSC(GSC_Final, GSC_Sub_Final, Concatenated = True)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
model = get_model(TrainingData.shape[0])
history = Neural_Net(TrainingData, TrainingTarget)

NN_Graph(history)
NN_Accuracy(TestData, TestDataAct)

GSC Dataset with feature concatenation
(940, 5000)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 256)               240896    
_________________________________________________________________
activation_19 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               131584    
_________________________________________________________________
activation_20 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 513       
_________________________________________________________________
activation_21 (Activation

3200/3200 [==============================] - 0s 122us/step - loss: 1.3999e-05 - acc: 1.0000 - val_loss: 0.1090 - val_acc: 0.9875
Epoch 50/1000
3200/3200 [==============================] - 0s 118us/step - loss: 8.5258e-06 - acc: 1.0000 - val_loss: 0.1159 - val_acc: 0.9875
Epoch 51/1000
3200/3200 [==============================] - 0s 98us/step - loss: 7.7165e-06 - acc: 1.0000 - val_loss: 0.1156 - val_acc: 0.9875
Epoch 52/1000
3200/3200 [==============================] - 0s 108us/step - loss: 6.8777e-06 - acc: 1.0000 - val_loss: 0.1174 - val_acc: 0.9875
Epoch 53/1000
3200/3200 [==============================] - 0s 109us/step - loss: 7.9532e-06 - acc: 1.0000 - val_loss: 0.1159 - val_acc: 0.9875
Epoch 54/1000
3200/3200 [==============================] - 0s 104us/step - loss: 1.0023e-05 - acc: 1.0000 - val_loss: 0.1134 - val_acc: 0.9875
Epoch 55/1000
3200/3200 [==============================] - 0s 99us/step - loss: 8.7530e-06 - acc: 1.0000 - val_loss: 0.1145 - val_acc: 0.9875
Epoch 56/1000
3

In [47]:
#GSC CONCATENATED
RawData, RawTarget = Select_GSC(GSC_Final, GSC_Sub_Final, Concatenated = False)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
model = get_model(TrainingData.shape[0])
history = Neural_Net(TrainingData, TrainingTarget)

NN_Graph(history)
NN_Accuracy(TestData, TestDataAct)

GSC Dataset with feature subtraction
(474, 5000)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 256)               121600    
_________________________________________________________________
activation_22 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 512)               131584    
_________________________________________________________________
activation_23 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 513       
_________________________________________________________________
activation_24 (Activation) 

3200/3200 [==============================] - 0s 102us/step - loss: 2.2735e-05 - acc: 1.0000 - val_loss: 0.0979 - val_acc: 0.9738
Epoch 50/1000
3200/3200 [==============================] - 0s 93us/step - loss: 2.1267e-05 - acc: 1.0000 - val_loss: 0.0980 - val_acc: 0.9738
Epoch 51/1000
3200/3200 [==============================] - 0s 88us/step - loss: 1.9535e-05 - acc: 1.0000 - val_loss: 0.0985 - val_acc: 0.9725
Epoch 52/1000
3200/3200 [==============================] - 0s 92us/step - loss: 1.8926e-05 - acc: 1.0000 - val_loss: 0.0985 - val_acc: 0.9738
Epoch 53/1000
3200/3200 [==============================] - 0s 98us/step - loss: 1.7124e-05 - acc: 1.0000 - val_loss: 0.0987 - val_acc: 0.9738
Epoch 54/1000
3200/3200 [==============================] - 0s 95us/step - loss: 1.7592e-05 - acc: 1.0000 - val_loss: 0.0988 - val_acc: 0.9738
Epoch 55/1000
3200/3200 [==============================] - 0s 119us/step - loss: 1.6096e-05 - acc: 1.0000 - val_loss: 0.0992 - val_acc: 0.9738
Epoch 56/1000
3200

3200/3200 [==============================] - 0s 110us/step - loss: 3.6286e-06 - acc: 1.0000 - val_loss: 0.1109 - val_acc: 0.9738
Epoch 107/1000
3200/3200 [==============================] - 0s 85us/step - loss: 3.6810e-06 - acc: 1.0000 - val_loss: 0.1108 - val_acc: 0.9738
Epoch 00107: early stopping


# LOGISTIC REGRESSION

In [62]:
def sigmoid(x): 
    return 1 / (1 + np.exp(-x))

def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

def GetAccuracy(W, ValData, ValDataAct):
    final_score = np.dot(ValData.T, W.T)
    #print(final_score.shape)

    preds = np.round(sigmoid(final_score))
    #print(preds[1])

    right = 0
    for i in range(ValDataAct.shape[0]):
        if preds[i] == ValDataAct[i]:
            right = right + 1
    #print(right)

    return (right/ValDataAct.shape[0])

def Logistic_Regression(TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct):
    W = np.zeros((1,TrainingData.shape[0]))
    LR = 0.2

    for i in range(10000):
        Z = np.dot(W, TrainingData)
        #print(Z.shape)

        y_cap = sigmoid(Z)
        #print(y_cap)

        delta_E = np.dot((y_cap - TrainingTarget), TrainingData.T)
        #print(delta_E.shape)

        #Update weights
        W = W -  np.dot(LR, delta_E)

        #if i%1000 == 0:
           # print("Loss %f"  %loss(y_cap,TrainingTarget))
    print(W.shape)

    Training_acc = GetAccuracy(W, TrainingData, TrainingTarget)
    Validation_acc = GetAccuracy(W, ValData, ValDataAct)
    Testing_acc = GetAccuracy(W, TestData, TestDataAct)
    #Unseen_acc = GetAccuracy(W, Unseen_GSC_Final[:, :1024], Unseen_GSC_Final[:, 1024:])
    print("Accuracy measures:")
    print("Training data accuracy", Training_acc)
    print("Validation data accuracy", Validation_acc)
    print("Test data accuracy", Testing_acc)

    #JUST TO VERIFY THE ACCURACY USING SKLEARN
    from sklearn.linear_model import LogisticRegression

    clf = LogisticRegression(fit_intercept=True, C = 1e15)
    clf.fit(TrainingData.T, TrainingTarget)
    print ('Accuracy from sk-learn on training:', clf.score(TrainingData.T, TrainingTarget))
    print ('Accuracy from sk-learn on validation:', clf.score(ValData.T, ValDataAct))
    print ('Accuracy from sk-learn on testing:', clf.score(TestData.T, TestDataAct))

In [63]:
#HUMAN OBSERVED DATASET CONCATENATED
RawData, RawTarget = Select_HODS(Final_HODS, Final_SHODS, Concatenated = True)
#print(RawData.shape, RawTarget.shape)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
Logistic_Regression(TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct)

Human Observed Dataset with Concatenation selected
(1582, 18)
(1, 18)
Accuracy measures:
Training data accuracy 0.9549763033175356
Validation data accuracy 0.9367088607594937
Test data accuracy 0.9554140127388535
Accuracy from sk-learn on training: 0.9605055292259084
Accuracy from sk-learn on validation: 0.9430379746835443
Accuracy from sk-learn on testing: 0.9490445859872612


In [64]:
#HUMAN OBSERVED DATASET SUBTRACTED
RawData, RawTarget = Select_HODS(Final_HODS, Final_SHODS, Concatenated = False)
#print(RawData.shape, RawTarget.shape)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
Logistic_Regression(TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct)

Human Observed Dataset with Subtraction selected
(1, 9)
Accuracy measures:
Training data accuracy 0.6216429699842022
Validation data accuracy 0.620253164556962
Test data accuracy 0.6751592356687898
Accuracy from sk-learn on training: 0.8199052132701422
Accuracy from sk-learn on validation: 0.7784810126582279
Accuracy from sk-learn on testing: 0.8535031847133758


In [66]:
#GSC CONCATENATED
RawData, RawTarget = Select_GSC(GSC_Final, GSC_Sub_Final, Concatenated = True)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
Logistic_Regression(TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct)

GSC Dataset with feature concatenation
(940, 5000)


KeyboardInterrupt: 

In [ ]:
#GSC SUBTRACTED
RawData, RawTarget = Select_GSC(GSC_Final, GSC_Sub_Final, Concatenated = False)
TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct = Generate_Data_Split(RawData, RawTarget)
Logistic_Regression(TrainingData, TrainingTarget, ValData, ValDataAct, TestData, TestDataAct)